In [2]:
import causaldag as cd
from causaldag.inference.structural import gsp, threshold_ug
from causaldag.utils.ci_tests import gauss_ci_test, MemoizedCI_Tester, dsep_test
from causaldag.utils.ci_tests import hsic_test
import numpy as np
from pprint import pprint
import random
from sklearn.covariance import GraphicalLassoCV
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

## Create random DAGs

In [3]:
nnodes = 10
sparsity = 2/(nnodes - 1)
nsamples = 10000
ndags = 50

np.random.seed(3881)
random.seed(1883)
dags = cd.rand.directed_erdos(nnodes, sparsity, ndags)
gdags = [cd.rand.rand_weights(dag) for dag in dags]
samples_list = [gdag.sample(nsamples) for gdag in gdags]
suffstats = [
    dict(C=np.corrcoef(samples, rowvar=False), n=nsamples) for samples in samples_list
]

## Algorithm settings

In [4]:
depth = 5
nruns = 10

## Estimate with GSP with random initialization

In [5]:
est_dags_random = []
for suffstat in suffstats:
    tester = MemoizedCI_Tester(gauss_ci_test, suffstat, alpha=.05)
    est_dags_random.append(gsp(nnodes, tester, depth=depth, nruns=nruns)[0])

## Estimate with GSP initializing based on undirected graph

In [6]:
est_dags_smart = []
for suffstat in suffstats:
    tester = MemoizedCI_Tester(gauss_ci_test, suffstat, alpha=.05)
    ug = threshold_ug(nnodes, tester)
    est_dags_smart.append(gsp(nnodes, tester, depth=depth, nruns=nruns, initial_undirected=ug)[0])

## Estimate with GSP initializing based on graphical lasso

In [7]:
est_dags_glasso = []
for suffstat in suffstats:
    tester = MemoizedCI_Tester(gauss_ci_test, suffstat, alpha=.05)
    gl = GraphicalLassoCV(cv=3)
    gl.fit(suffstat['C'])
    edges = {(i, j) for i, j in zip(*gl.precision_.nonzero()) if i != j}
    ug = cd.UndirectedGraph(nodes=set(range(nnodes)), edges=edges)
    est_dags_glasso.append(gsp(nnodes, tester, depth=depth, nruns=nruns, initial_undirected=ug)[0])

# Compare results

In [8]:
shd_skeleton_random = [
    dag.shd_skeleton(est_dag) for dag, est_dag in zip(dags, est_dags_random)
]
shd_skeleton_smart = [
    dag.shd_skeleton(est_dag) for dag, est_dag in zip(dags, est_dags_smart)
]
shd_skeleton_glasso = [
    dag.shd_skeleton(est_dag) for dag, est_dag in zip(dags, est_dags_glasso)
]
print(np.mean(shd_skeleton_random))
print(np.mean(shd_skeleton_smart))
print(np.mean(shd_skeleton_glasso))

2.44
2.44
3.52


In [9]:
shd_cpdag_random = [
    dag.cpdag().shd(est_dag.cpdag()) for dag, est_dag in zip(dags, est_dags_random)
]
shd_cpdag_smart = [
    dag.cpdag().shd(est_dag.cpdag()) for dag, est_dag in zip(dags, est_dags_smart)
]
shd_cpdag_glasso = [
    dag.cpdag().shd(est_dag.cpdag()) for dag, est_dag in zip(dags, est_dags_glasso)
]
print(np.mean(shd_cpdag_random))
print(np.mean(shd_cpdag_smart))
print(np.mean(shd_cpdag_glasso))

3.92
4.06
5.56


In [10]:
markov_equivalent_random = [
    dag.markov_equivalent(est_dag) for dag, est_dag in zip(dags, est_dags_random)
]
markov_equivalent_smart = [
    dag.markov_equivalent(est_dag) for dag, est_dag in zip(dags, est_dags_smart)
]
markov_equivalent_glasso = [
    dag.markov_equivalent(est_dag) for dag, est_dag in zip(dags, est_dags_glasso)
]
print(np.mean(markov_equivalent_random))
print(np.mean(markov_equivalent_smart))
print(np.mean(markov_equivalent_glasso))

0.08
0.08
0.08


# Oracle results

In [11]:
est_dags_oracle = []
for dag in dags:
    tester = MemoizedCI_Tester(dsep_test, dag)
    ug = threshold_ug(nnodes, tester)
    est_dags_oracle.append(gsp(nnodes, tester, depth=depth, nruns=nruns, initial_undirected=ug)[0])

In [12]:
shd_skeleton_oracle = [
    dag.shd_skeleton(est_dag) for dag, est_dag in zip(dags, est_dags_oracle)
]
shd_cpdag_oracle = [
    dag.cpdag().shd(est_dag.cpdag()) for dag, est_dag in zip(dags, est_dags_oracle)
]
markov_equivalent_oracle = [
    dag.markov_equivalent(est_dag) for dag, est_dag in zip(dags, est_dags_oracle)
]
print(np.mean(shd_skeleton_oracle))
print(np.mean(shd_cpdag_oracle))
print(np.mean(markov_equivalent_oracle))

0.0
0.0
1.0
